In [22]:
import json, os
import pandas as pd

In [31]:
# Specify the test data filepath
s3_path = os.path.join('test_data', 'allplayers.json')
jsonfile = open(s3_path)
contents = json.load(jsonfile)
# Convert JSON to dataframe
allplayers = pd.DataFrame.from_dict(contents)
# Transpose dataframe
allplayers = allplayers.T
# Create a column for id_sleeper based on the index
allplayers = allplayers.reset_index(names='id_sleeper')
# Select only relevant columns
allplayers = allplayers[[
    'id_sleeper',
    'full_name', 
    'weight', 'height',
    'birth_date', 'age', 
    'high_school', 'college',
    'sport', 'years_exp', 'active', 'status',
    'team', 'number', 'position', 'fantasy_positions', 'depth_chart_position', 'depth_chart_order',
    'news_updated', 'injury_status', 'injury_body_part', 'injury_start_date', 'injury_notes', 'practice_description', 'practice_participation',
    'competitions',
]]

In [32]:
# Read static data
ids = pd.read_csv('test_data/lu_ids.csv')
# Read other scraped data
ourlads = pd.read_csv('test_data/ourlads.csv')
sharks = pd.read_csv('test_data/sharks.csv', dtype={'Week':str})

In [33]:
#sharks['id_sharks'] = sharks['id_sharks'].apply(lambda x: ' '.join(x.split(', ')[::-1]))


In [34]:
# Create a scoring dataset
scoring = ids.merge(
    sharks, how='inner', on='id_sharks'
).merge(
    ourlads, how = 'left', on='id_ourlads'
)
# Clean column names
scoring = scoring.drop(columns=[
    'id_ourlads', 'id_sharks', '#', 'Tm', 'Opp',
])

# Create a primary key
scoring['index_scoring'] = scoring['id_sleeper'] + "_" + scoring['Week']
# Drop NA values
scoring = scoring.dropna(subset='index_scoring')
# Drop duplicates
scoring = scoring.drop_duplicates(subset='index_scoring')

,id_sleeper,Att,Comp,Pass Yds,Pass TDs,0-9 Pass TDs,10-19 Pass TDs,20-29 Pass TDs,30-39 Pass TDs,40-49 Pass TDs,...,Punt Yds,Punts Inside 20,Yds Allowed,Pts Agn,Scks,DefTD,Safts,PR,KR,index_scoring
8570,8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8122_13
8571,8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8122_14
8572,8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8122_15
8573,8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8122_16
8574,8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8122_17


In [37]:
# Derive additional columns from others
scoring.loc[scoring['PR']==True, 'pr_yd'] = 13
scoring.loc[scoring['KR']==True, 'kr_yd'] = 19
scoring['fgm_yds_over_30'] = scoring['30-39 FGM'] + scoring['40-49 FGM'] + scoring['50+ FGM']
scoring.loc[(scoring['Pts Agn']<1), 'pts_allow_0'] = 1
scoring.loc[(scoring['Pts Agn']>=1) & (scoring['Pts Agn']<7), 'pts_allow_1_6'] = 1
scoring.loc[(scoring['Pts Agn']>=7) & (scoring['Pts Agn']<14), 'pts_allow_7_13'] = 1
scoring.loc[(scoring['Pts Agn']>=14) & (scoring['Pts Agn']<21), 'pts_allow_14_20'] = 1
scoring.loc[(scoring['Pts Agn']>=21) & (scoring['Pts Agn']<28), 'pts_allow_21_27'] = 1
scoring.loc[(scoring['Pts Agn']>=28) & (scoring['Pts Agn']<35), 'pts_allow_28_34'] = 1
scoring.loc[(scoring['Pts Agn']>=35), 'pts_allow_35p'] = 1
scoring.loc[(scoring['Yds Allowed']<100), 'yds_allow_0_100'] = 1
scoring.loc[(scoring['Yds Allowed']>=100) & (scoring['Yds Allowed']<200), 'yds_allow_100_199'] = 1
scoring.loc[(scoring['Yds Allowed']>=200) & (scoring['Yds Allowed']<300), 'yds_allow_200_299'] = 1
scoring.loc[(scoring['Yds Allowed']>=400) & (scoring['Yds Allowed']<450), 'yds_allow_400_449'] = 1
scoring.loc[(scoring['Yds Allowed']>=450) & (scoring['Yds Allowed']<500), 'yds_allow_450_499'] = 1
scoring.loc[(scoring['Yds Allowed']>=500) & (scoring['Yds Allowed']<550), 'yds_allow_500_549'] = 1
scoring.loc[(scoring['Yds Allowed']>=550), 'yds_allow_550p'] = 1
scoring.loc[(scoring['Rsh Yds']>=100) & (scoring['Rsh Yds']<200), 'bonus_rush_yd_100'] = 1
scoring.loc[scoring['Rsh Yds']>=200, 'bonus_rush_yd_200'] = 1
scoring.loc[(scoring['Rec Yds']>=100) & (scoring['Rec Yds']<200), 'bonus_rec_yd_100'] = 1
scoring.loc[scoring['Rec Yds']>=200, 'bonus_rec_yd_200'] = 1
scoring.loc[(scoring['Pass Yds']>=300) & (scoring['Pass Yds']<400), 'bonus_pass_yd_300'] = 1
scoring.loc[scoring['Pass Yds']>=400, 'bonus_pass_yd_400'] = 1
scoring['Rush and Rec Yds'] = scoring['Rsh Yds'] + scoring['Rec Yds']
scoring.loc[scoring['Rush and Rec Yds']>=200, 'bonus_rush_rec_yd_200'] = 1

# Rename columns
scoring = scoring.rename(columns={
    'Week':'week_of_season',
    'Comp':'pass_cmp', 'Pass Yds':'pass_yd', 'Pass TDs':'pass_td', 
    'Int':'pass_int', 
    'Rush':'rush_att', 'Rsh Yds':'rush_yd', 'Rsh TDs':'rush_td', 
    'Fum':'fum_lost',
    'Rec':'rec','Rec Yds':'rec_yd', 'Rec TDs':'rec_td', 
    'XPM':'xpm', 'FGM':'fgm', '10-19 FGM':'fgm_0_19','20-29 FGM':'fgm_20_29', '30-39 FGM':'fgm_30_39', '40-49 FGM':'fgm_40_49', '50+ FGM':'fgm_50p',
    'Miss':'fgmiss', 
    'Scks':'sack', 'DefTD':'def_st_td', 'Safts':'safe',     
})
# Drop unnecessary columns which sharks has but sleeper lacks
scoring = scoring.drop(columns=[
    'Att', '0-9 Pass TDs', '10-19 Pass TDs', '20-29 Pass TDs', '30-39 Pass TDs', '40-49 Pass TDs', '50+ Pass TDs', 'Sck', 
    '0-9 Rsh TDs', '10-19 Rsh TDs', '20-29 Rsh TDs', '30-39 Rsh TDs', '40-49 Rsh TDs', '50+ Rsh TDs',
    '>= 50 yd', '>= 100 yd','0-9 Rec TDs', '10-19 Rec TDs', '20-29 Rec TDs', '30-39 Rec TDs','40-49 Rec TDs', '50+ Rec TDs',
    'Tgt', 'RZ Tgt', 
    'Kick Ret Yds','PR', 'KR',
    'XPA','FGA', 
    'Punts','Punt Yds', 'Punts Inside 20', 
    'Yds Allowed', 'Pts Agn', 
])
# Set to zero predictions which sharks lacks but sleeper has
for colName in [
    'pass_2pt', 'rush_2pt', 'rec_2pt', 'xpmiss', 
    'int', 'fum_rec', 'blk_kick', 'ff','def_st_ff','def_st_fum_rec',
    'def_td','def_3_and_out','def_2pt',
    'st_fum_rec','st_ff','st_td',
    'fum','fum_rec_td',
]:
    scoring[colName] = 0
# Set NA values to zero
scoring = scoring.fillna(0)


/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_67281/2869031133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoring.loc[scoring['PR']==True, 'pr_yd'] = 13


In [38]:
# Tidy up columns
scoring = scoring[[
    'index_scoring', 'id_sleeper', 'week_of_season', 'pass_cmp', 'pass_yd',
    'pass_td', 'pass_int', 'rush_att', 'rush_yd', 'rush_td', 'fum_lost',
    'rec', 'rec_yd', 'rec_td', 'xpm', 'fgm', 'fgm_0_19', 'fgm_20_29',
    'fgm_30_39', 'fgm_40_49', 'fgm_50p', 'fgmiss', 'sack', 'def_st_td',
    'safe', 'pr_yd', 'kr_yd', 'fgm_yds_over_30', 'pts_allow_0',
    'pts_allow_1_6', 'pts_allow_7_13', 'pts_allow_14_20', 'pts_allow_21_27',
    'pts_allow_28_34', 'pts_allow_35p', 'yds_allow_0_100',
    'yds_allow_100_199', 'yds_allow_200_299', 'yds_allow_400_449',
    'yds_allow_450_499', 'yds_allow_500_549', 'yds_allow_550p',
    'bonus_rush_yd_100', 'bonus_rush_yd_200', 'bonus_rec_yd_100',
    'bonus_rec_yd_200', 'bonus_pass_yd_300', 'bonus_pass_yd_400',
    'Rush and Rec Yds', 'bonus_rush_rec_yd_200', 'pass_2pt', 'rush_2pt',
    'rec_2pt', 'xpmiss', 'int', 'fum_rec', 'blk_kick', 'ff', 'def_st_ff',
    'def_st_fum_rec', 'def_td', 'def_3_and_out', 'def_2pt', 'st_fum_rec',
    'st_ff', 'st_td', 'fum', 'fum_rec_td'
]]

In [39]:
scoring.columns

Index(['index_scoring', 'id_sleeper', 'week_of_season', 'pass_cmp', 'pass_yd',
       'pass_td', 'pass_int', 'rush_att', 'rush_yd', 'rush_td', 'fum_lost',
       'rec', 'rec_yd', 'rec_td', 'xpm', 'fgm', 'fgm_0_19', 'fgm_20_29',
       'fgm_30_39', 'fgm_40_49', 'fgm_50p', 'fgmiss', 'sack', 'def_st_td',
       'safe', 'pr_yd', 'kr_yd', 'fgm_yds_over_30', 'pts_allow_0',
       'pts_allow_1_6', 'pts_allow_7_13', 'pts_allow_14_20', 'pts_allow_21_27',
       'pts_allow_28_34', 'pts_allow_35p', 'yds_allow_0_100',
       'yds_allow_100_199', 'yds_allow_200_299', 'yds_allow_400_449',
       'yds_allow_450_499', 'yds_allow_500_549', 'yds_allow_550p',
       'bonus_rush_yd_100', 'bonus_rush_yd_200', 'bonus_rec_yd_100',
       'bonus_rec_yd_200', 'bonus_pass_yd_300', 'bonus_pass_yd_400',
       'Rush and Rec Yds', 'bonus_rush_rec_yd_200', 'pass_2pt', 'rush_2pt',
       'rec_2pt', 'xpmiss', 'int', 'fum_rec', 'blk_kick', 'ff', 'def_st_ff',
       'def_st_fum_rec', 'def_td', 'def_3_and_out', 'def_2pt